Lyrics & an Interesting Application of Transformers
===
MAIC - Winter, Week 7<br>
```
  _____________
 /0   /     \  \
/  \ M A I C/  /\
\ / *      /  / /
 \___\____/  @ /
          \_/_/
```

**The dataset we're using:**

[https://www.kaggle.com/datasets/mervedin/genius-lyrics]

---
# Data Stuff

In [ ]:
# ML imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Data-handling imports
import numpy as np
import pandas as pd

# String-handling
import re

# Display-handling imports
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [ ]:
def clean_lyrics(lyrics_raw):
    verses = []
    
    # extract & clean verses
    for song in lyrics_raw:
        song = '\n'.join(song.splitlines()[1:]) # remove first line from song -- it's just song name
        song = re.sub('\\d+Embed', '', song) # songs have a weird string at the end "<numbers>Embed"
        song = song.replace('in\'', 'ing') # in' to ing
        song = re.sub('\\[.*\]|\\(.*\\)', '', song).lower() # remove things surounded by parens or brackets
        song = re.sub('[^a-z\'\\n ]', ' ', song) # remove non-letters, non-quotes, non-newlines, non-spaces
        verse = re.split('\n\n', song) # split verses via double newline
        verse = list(map(lambda s: s.strip(), verse)) # remove extra surounding newlines
        verses += verse
    
    return verses

#Tokenization?
# "the" -> 3
# "of" -> 4
# ...
# "nondeterministically" -> 13044

# BUT ALSO;

# <start> -> 1
# <end> -> 2
# <nothing> -> 0

START_TOK = 1
END_TOK = 2
word_to_idx = {'0':0, 'START':START_TOK, 'END':END_TOK} # 0 = no word
vocab_size = len(word_to_idx)

def tokenize_verse(verse):
    def word_to_idx_emplace(word):
        global vocab_size
        if word not in word_to_idx:
            word_to_idx[word] = vocab_size
            vocab_size += 1
        return word_to_idx[word]
    
    return list(map(word_to_idx_emplace, verse.split()))

verses = clean_lyrics(pd.read_csv('./Imagine-Dragons.csv').dropna()['lyrics']) # IMPORT RAW LYRICS
verses_tokenized = [tokenize_verse(v) for v in verses] #tokenize

idx_to_word = {v: k for k, v in word_to_idx.items()}

In [ ]:
print([idx_to_word[i] for i in verses_tokenized[44]])
print(verses_tokenized[44])

In [ ]:
max_len = 0
for v in verses_tokenized:
    max_len = max(max_len, len(v))
max_len += 2 # PADDING for eventual START and END tokens during training
max_len

In [ ]:
# PADDING
for i in range(len(verses_tokenized)):
    l = len(verses_tokenized[i])
    verses_tokenized[i] += [0]*(max_len-l)

In [ ]:
verses_tokenized = np.array(verses_tokenized)
print(
    verses_tokenized, '\n\n',
    verses_tokenized.shape
)

---
# Model Stuff

In [ ]:
# review attention, self-attention, multi-headed
t = np.array([
    [0.1,0.1,0.8,0.0],
    [0.0,0.0,0.5,0.5]
    ])[np.newaxis,:]
layers.MultiHeadAttention(1,4)(t, t)

In [ ]:
# How does this map to the transformer diagram?

class DecoderBlock(layers.Layer):
    def __init__(self,
            *,
            embed_dim,
            num_heads,
            ff_dim,
            dropout_rate=0.1):
        super(DecoderBlock, self).__init__()

        self.masked_self_attention = layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=embed_dim,
        dropout=dropout_rate)

        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)
        
        self.ffn = layers.Dense(ff_dim, activation='relu')
        self.ffn_to_embedded = layers.Dense(embed_dim)

    def call(self, x, mask, training):
        attn_out = self.masked_self_attention(query=x, value=x, key=x, training=training, attention_mask=mask)
        attn_out = self.dropout1(attn_out, training=training)
        x = self.layernorm1(attn_out + x)

        ffn_out = self.ffn(x)
        ffn_out = self.ffn_to_embedded(ffn_out)
        ffn_out = self.dropout2(ffn_out, training=training)
        return self.layernorm2(ffn_out + x)

In [ ]:
#  This ALSO maps to the transformer diagram

embed_dim = 32  # Embedding size for each token
num_heads = 6  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input((max_len,))

token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
positions = tf.range(start=0, limit=max_len, delta=1)
# easier than sinusoidal and has similar results (according to the transformer paper)
pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)(positions)
x = token_emb + pos_emb

mask = layers.Lambda(lambda x: x != 0)(inputs)

x = DecoderBlock(embed_dim=embed_dim, num_heads=num_heads, ff_dim=ff_dim)(x, mask) # you can stack these

x = layers.Dense(vocab_size, activation=None)(x) # logits

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

---
# Train Stuff

In [ ]:
# This is also from the transformer paper

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

learning_rate = CustomSchedule(embed_dim)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

# optimizer = tf.keras.optimizers.Adam(1e-3, beta_1=0.9, beta_2=0.98,
#                                      epsilon=1e-9)

In [ ]:
losses = []

# Training Process?

def train_ittr(verse, disp_prefix):
    global losses
    
    target_seq = np.copy(verse)
    target_seq[np.argmin(target_seq)] = END_TOK
    
    input_seq = np.concatenate(([START_TOK],target_seq))[:-1]
    
    idx = 2
    while True:
        # get tokens into trainable masked form
        target_seq_masked = np.copy(target_seq)
        input_seq_masked = np.copy(input_seq)
        target_seq_masked[idx:] = 0
        input_seq_masked[idx:] = 0
        
        if input_seq_masked[idx-1] == END_TOK: break
        
        target_seq_masked = np.eye(vocab_size)[target_seq_masked]
        
        # backprop
        with tf.GradientTape() as tape:
            pred = model(input_seq_masked[np.newaxis,:])[0]
            
            # for showing the data
#             if idx==5:
#                 print(input_seq_masked[:idx])
#                 print(target_seq_masked[:idx])
#                 print(pred[:idx])
#                 return
            
            mask = input_seq_masked[np.newaxis,:] != 0
            loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
            loss = loss_object(np.argmax(target_seq_masked, axis=1), pred)

            mask = tf.cast(mask, dtype=loss.dtype)
            loss *= mask

            loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
            
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        losses += [np.array(loss)]
        clear_output(wait=True)
        print('Seq Position:', idx)
        print(disp_prefix, 'loss:',np.array(loss))
        
        idx+=1

In [ ]:
%%time

losses = []
epochs = 1

for i in range(epochs):
    verse_idx = 0
    for verse in verses_tokenized:
        verse_idx += 1
        train_ittr(verse, disp_prefix=f'verse: {verse_idx} | epoch: {i+1} |')

In [ ]:
losses = np.array(losses)
plt.plot(np.arange(0, losses.shape[0]), losses)

# Why does the loss look so bad?

---
# Prediction

In [ ]:
verses[10] # for reference

In [ ]:
test_str = "just a young gun"
test_str = test_str.split()
test_str = list(map(lambda s: word_to_idx[s], test_str))
test_str = [START_TOK] + test_str
l = len(test_str)
test_str += [0]*(max_len-l)

for i in range(max_len-l):
    clear_output(wait=True)
    print(i)
    pred = model.predict(np.array(test_str)[np.newaxis, :])
    pred_idxs = np.array(tf.argmax(tf.nn.softmax(pred)[0], axis=1))
    test_str[l] = pred_idxs[l-1]
    l += 1

print(
    ' '.join(
        list(map(
            lambda i: idx_to_word[i],
            test_str
        ))
    )
)

# What patterns emerge?

---

**Resources**

- Attention is All You Need ("The Transformer Paper") [https://arxiv.org/pdf/1706.03762.pdf]
- Simple Sentiment Classification w/ transformers (an easy project to try) [https://keras.io/examples/nlp/text_classification_with_transformer/]
- More complex and in-depth attention tutorial [https://www.tensorflow.org/text/tutorials/transformer]

---

**Challenge Problems**

<span style="color:green">&#9646;&#9646;</span><br>
(1 challenge point)<br>
Add more decoder stacks to the model.<br>
<br><br>
<span style="color:green">&#9646;&#9646;</span><br>
(1 challenge point)<br>
Train the model and a different set of lyrics.<br>
<br><br>
<span style="color:green">&#9646;&#9646;</span><span style="color:#eedd00">&#9646;&#9646;</span><br>
(2 challenge points)<br>
Train the model on an aggregated set of lyrics from many artists. (Also counts for prior challenge problem)<br>
<br><br>
<span style="color:green">&#9646;&#9646;</span><span style="color:#eedd00">&#9646;&#9646;</span><span style="color:red">&#9646;&#9646;</span><br>
(3 challenge points)<br>
Use batches to train the model faster and stablize the loss. (Hint: train_itter can be made into a training-data-generation function)<br>
<br><br>
<span style="color:green">&#9646;&#9646;</span><span style="color:#eedd00">&#9646;&#9646;</span><span style="color:red">&#9646;&#9646;</span><span style="color:magenta">&#9646;&#9646;</span><br>
(4 challenge points)<br>
Implement and incorporate a tranformer-encoder into this model.<br>